# Maddison Database Project

In [1]:
import pandas as pd
from pathlib import Path

#Loading MPD 2020
file = Path('data/mpd2020.xlsx')
mpd = pd.read_excel(file, sheet_name='Full data')

#Loading MPD 2020 - Regional data
file = Path('data/mpd2020.xlsx')
mpd_regional = pd.read_excel(file, sheet_name='Regional data', header=1)

#Loading Regional composition
file = Path('data/Regional composition_web.xlsx')
mpd_regionalcomp = pd.read_excel(file, sheet_name='Sheet2')

In [2]:
mpd

,countrycode,country,year,gdppc,pop
0,AFG,Afghanistan,1820,NaN,3280.00000
1,AFG,Afghanistan,1870,NaN,4207.00000
2,AFG,Afghanistan,1913,NaN,5730.00000
3,AFG,Afghanistan,1950,1156.0000,8150.00000
4,AFG,Afghanistan,1951,1170.0000,8284.00000
...,...,...,...,...,...
21677,ZWE,Zimbabwe,2014,1594.0000,13313.99205
21678,ZWE,Zimbabwe,2015,1560.0000,13479.13812
21679,ZWE,Zimbabwe,2016,1534.0000,13664.79457
21680,ZWE,Zimbabwe,2017,1582.3662,13870.26413


In [3]:
mpd_regional = mpd_regional.rename(columns={"Region": "Year"})
mpd_regional = mpd_regional.drop(index=0).reset_index(drop=True)
mpd_regional

,Year,Western Europe,Eastern Europe,Western Offshoots,Latin America,Asia (East),Asia (South and South-East),Middle East,Sub-Sahara Africa,Western Europe.1,...,Eastern Europe.1,Latin America.1,Asia (South and South-East).1,Asia (East).1,Middle East.1,Sub-Sahara Africa.1,World,World GDP pc,Unnamed: 19,Unnamed: 20
0,1820,2306.966690,818.000000,2513.045418,952.816821,1088.584951,929.000000,974.000000,800.000000,132371.000000,...,90785.000000,20099.000000,2.556950e+05,4.277570e+05,35600.000000,6.000000e+04,1.033538e+06,1101.565459,NaN,NaN
1,1830,2384.314573,942.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1840,2579.534884,907.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1850,2678.000000,985.000000,3474.410149,1080.657319,899.657658,929.000000,1000.000000,800.000000,165348.000000,...,117661.000000,29485.000000,2.787060e+05,4.557740e+05,42000.000000,6.500000e+04,1.180734e+06,1225.081067,NaN,NaN
4,1860,3034.000000,1358.000000,4214.440703,1588.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1870,3301.304919,1575.000000,4647.453901,1318.541091,989.477165,850.346572,1165.000000,800.000000,186566.000000,...,141626.000000,37692.000000,3.276920e+05,4.091530e+05,49839.000000,7.000000e+04,1.268656e+06,1497.982325,NaN,NaN
6,1880,3585.154363,1886.000000,6019.123928,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1890,4079.313847,2204.000000,6480.955612,1672.812962,NaN,950.999236,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1900,4724.295785,2700.000000,7740.850451,1750.626743,1086.328445,994.419007,1300.000000,850.000000,232433.000000,...,194693.000000,61004.000000,3.660600e+05,4.570580e+05,56000.000000,8.600000e+04,1.539644e+06,2212.043316,NaN,NaN
9,1910,5134.759064,2282.571503,9354.690921,2193.539665,NaN,1143.117595,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
mpd_regional

,Year,Western Europe,Eastern Europe,Western Offshoots,Latin America,Asia (East),Asia (South and South-East),Middle East,Sub-Sahara Africa,Western Europe.1,...,Eastern Europe.1,Latin America.1,Asia (South and South-East).1,Asia (East).1,Middle East.1,Sub-Sahara Africa.1,World,World GDP pc,Unnamed: 19,Unnamed: 20
0,1820,2306.966690,818.000000,2513.045418,952.816821,1088.584951,929.000000,974.000000,800.000000,132371.000000,...,90785.000000,20099.000000,2.556950e+05,4.277570e+05,35600.000000,6.000000e+04,1.033538e+06,1101.565459,NaN,NaN
1,1830,2384.314573,942.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1840,2579.534884,907.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1850,2678.000000,985.000000,3474.410149,1080.657319,899.657658,929.000000,1000.000000,800.000000,165348.000000,...,117661.000000,29485.000000,2.787060e+05,4.557740e+05,42000.000000,6.500000e+04,1.180734e+06,1225.081067,NaN,NaN
4,1860,3034.000000,1358.000000,4214.440703,1588.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1870,3301.304919,1575.000000,4647.453901,1318.541091,989.477165,850.346572,1165.000000,800.000000,186566.000000,...,141626.000000,37692.000000,3.276920e+05,4.091530e+05,49839.000000,7.000000e+04,1.268656e+06,1497.982325,NaN,NaN
6,1880,3585.154363,1886.000000,6019.123928,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1890,4079.313847,2204.000000,6480.955612,1672.812962,NaN,950.999236,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1900,4724.295785,2700.000000,7740.850451,1750.626743,1086.328445,994.419007,1300.000000,850.000000,232433.000000,...,194693.000000,61004.000000,3.660600e+05,4.570580e+05,56000.000000,8.600000e+04,1.539644e+06,2212.043316,NaN,NaN
9,1910,5134.759064,2282.571503,9354.690921,2193.539665,NaN,1143.117595,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
mpd_regionalcomp

,country,year,region
0,ITA,1820,Western Europe
1,NLD,1820,Western Europe
2,PRT,1820,Western Europe
3,SWE,1820,Western Europe
4,GBR,1820,Western Europe
...,...,...,...
299,ZWE,1950,Sub Saharan Africa
300,AUS,1950,Western Offshoots
301,CAN,1950,Western Offshoots
302,NZL,1950,Western Offshoots


In [6]:
mpd_regionalcomp1950 = mpd_regionalcomp[mpd_regionalcomp['year'] == 1950].copy().reset_index(drop=True)
mpd_regionalcomp1950

,country,year,region
0,AUT,1950,Western Europe
1,BEL,1950,Western Europe
2,DNK,1950,Western Europe
3,FIN,1950,Western Europe
4,FRA,1950,Western Europe
...,...,...,...
130,ZWE,1950,Sub Saharan Africa
131,AUS,1950,Western Offshoots
132,CAN,1950,Western Offshoots
133,NZL,1950,Western Offshoots


In [7]:
mpd_withregions = pd.merge(mpd, mpd_regionalcomp1950[['country','region']], left_on=['countrycode'], right_on=['country'], how='left', validate='many_to_many')
mpd_withregions

,countrycode,country_x,year,gdppc,pop,country_y,region
0,AFG,Afghanistan,1820,NaN,3280.00000,AFG,South and South East Asia
1,AFG,Afghanistan,1870,NaN,4207.00000,AFG,South and South East Asia
2,AFG,Afghanistan,1913,NaN,5730.00000,AFG,South and South East Asia
3,AFG,Afghanistan,1950,1156.0000,8150.00000,AFG,South and South East Asia
4,AFG,Afghanistan,1951,1170.0000,8284.00000,AFG,South and South East Asia
...,...,...,...,...,...,...,...
21791,ZWE,Zimbabwe,2014,1594.0000,13313.99205,ZWE,Sub Saharan Africa
21792,ZWE,Zimbabwe,2015,1560.0000,13479.13812,ZWE,Sub Saharan Africa
21793,ZWE,Zimbabwe,2016,1534.0000,13664.79457,ZWE,Sub Saharan Africa
21794,ZWE,Zimbabwe,2017,1582.3662,13870.26413,ZWE,Sub Saharan Africa


In [14]:
mpd_nullregions = mpd_withregions[mpd_withregions['region'].isnull()].copy().reset_index(drop=True)
mpd_nullregions

,countrycode,country_x,year,gdppc,pop,country_y,region
0,ALB,Albania,1,NaN,200.00000,NaN,NaN
1,ALB,Albania,1000,NaN,200.00000,NaN,NaN
2,ALB,Albania,1500,NaN,200.00000,NaN,NaN
3,ALB,Albania,1600,NaN,200.00000,NaN,NaN
4,ALB,Albania,1700,NaN,300.00000,NaN,NaN
...,...,...,...,...,...,...,...
2730,YUG,Former Yugoslavia,2014,14627.0000,21946.00000,NaN,NaN
2731,YUG,Former Yugoslavia,2015,14971.0000,21882.00000,NaN,NaN
2732,YUG,Former Yugoslavia,2016,15416.0000,21817.00000,NaN,NaN
2733,YUG,Former Yugoslavia,2017,15960.8432,21481.76458,NaN,NaN


In [13]:
mpd_nullregions['country_x'].unique()

array(['Albania', 'Armenia', 'Azerbaijan', 'Bangladesh',
       'Bosnia and Herzegovina', 'Belarus', "Côte d'Ivoire", 'Cyprus',
       'Czech Republic', 'Estonia', 'Gabon', 'Georgia', 'Gambia',
       'Croatia', 'Ireland', 'Kazakhstan', 'Kyrgyzstan', 'Lithuania',
       'Latvia', 'Republic of Moldova', 'TFYR of Macedonia', 'Malta',
       'Montenegro', 'D.P.R. of Korea', 'State of Palestine',
       'Russian Federation', 'Serbia', 'Former USSR', 'Slovakia',
       'Slovenia', 'Tajikistan', 'Turkmenistan', 'Ukraine', 'Uzbekistan',
       'Former Yugoslavia'], dtype=object)

In [15]:
len(mpd_nullregions['country_x'].unique())

35